*Ising Model Approach*

In [70]:
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from collections import Counter
collect_states_ising = Counter()


Matrix of weights and vector of biases.

In [30]:
matrix = [
    [0, -1, 2],
    [-1, 0, 2],
    [2, 2, 0]
]

weights = torch.tensor(matrix, dtype=torch.float32)
bias = torch.tensor([1, 1, -2], dtype=torch.float32)

In [32]:

def from_torch_to_numpy(tensor: torch.Tensor):
    return tensor.detach().cpu().numpy()

Implementation of the energy formula using numpy library.   

In [33]:
def energy_function(p_bits: torch.Tensor, weights: torch.Tensor, bias: torch.Tensor):
    return -np.sum(np.multiply(from_torch_to_numpy(weights), from_torch_to_numpy(torch.outer(p_bits, p_bits)))) - torch.dot(p_bits, bias)

In [34]:
def activation_function(syn_input: torch.Tensor, beta: float):
    return torch.heaviside(torch.tanh(beta * (-syn_input)) - (torch.rand(1) * 2 - 1), torch.tensor(0.0))

In [35]:
def calculate_gradient(p_bits: torch.Tensor, weights: torch.Tensor, bias: torch.Tensor):
    energy = energy_function(p_bits, weights, bias)
    energy.backward(torch.ones(energy.shape))
    p_bits.retain_grad()
    return p_bits.grad

In [62]:
def evaluate(beta, weights: torch.Tensor, bias: torch.Tensor):

    def model():
        
        p_bits = torch.tensor([0.0, 1.0, 1.0], requires_grad=True) # third bit is clamped to 1
        for i in range(1000):
            indices = torch.randperm(2)
            for p_bit_index in indices:
                gradient = calculate_gradient(p_bits, weights, bias)
                syn_input = gradient[p_bit_index.item()]
                with torch.no_grad():
                    p_bits.data[p_bit_index] = activation_function(syn_input, beta)
                collect_states_ising[(p_bits[0].item(), p_bits[1].item())] += 1
    model()

In [71]:
seed = 42
torch.manual_seed(seed)
evaluate(2 * 10 ** (-5), weights, bias)
print(collect_states_ising.most_common()[0])
collect_states_ising.clear()

((1.0, 1.0), 527)
